In [1]:
# Import modules
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import preprocessing

In [44]:
final = pd.read_csv('../../data/test_clean.csv', index_col=0)
train = pd.read_csv('../../data/train_clean.csv', index_col=0)

In [30]:
final['Age'] = 2019 - final['YearBuilt']
final_cont = final[['LotArea','Age','GarageArea']]
final_categ = final[['BsmtFullBath','KitchenAbvGr','GarageType','Kitchen','Fireplace','ExterQ',
                    'BsmtQ','HeatingQ','n_toilets']] #remove n_showers

In [47]:
train['Age'] = 2019 - train['YearBuilt']
train_cont = train[['LotArea','Age','GarageArea']]

In [31]:
# boxcox and normalize dataframes with sklearn.stats
def make_boxed_normal(df):
    new_dict = {}
    logs = []
    devs = []
    means = []
    cols = df.columns
    for name in cols: 
        key = 'norm_' + name
        arr = np.array(df[name]) + 1
        arr, log_num = stats.boxcox(arr)
        upper = arr.max()
        lower = arr.min()
        value = (arr-lower)/(upper-lower)
        new_dict[key] = value
        logs.append(log_num)
        devs.append(np.std(arr))
        means.append(np.mean(arr))
        
    return pd.DataFrame(new_dict), logs, devs, means

In [48]:
train_norm, logs,devs,means = make_boxed_normal(train_cont)

In [69]:
#We will pass into here the test data + log_list with lambdas from training data

def normalize_test(df, mean_arr, std_arr, log_list):
    new_dict = {}
    cols = df.columns
    i = 0
    for name in cols: 
        key = 'test_norm_' + name
        arr = np.array(df[name]) + 1 
        tempnum = log_list[i]
        print(tempnum)
        arr = stats.boxcox(arr, lmbda = tempnum)
        upper = arr.max()
        lower = arr.min()
        value = (arr-mean_arr[i])/(std_arr[i])
        print('Iter=',i)
        new_dict[key] = value
        i += 1
     
    return pd.DataFrame(new_dict)

In [73]:
final_norm = normalize_test(final_cont,means,devs,logs)

0.39726927868822215
Iter= 0
0.22006264293977928
Iter= 1
0.8207705726888868
Iter= 2


In [74]:
final_combo = pd.concat([final_norm,final_categ],axis=1)

In [75]:
x = final['GrLivArea'] 
mnGrLivArea = np.log(train.GrLivArea).mean()
stdGrLivArea = np.log(train.GrLivArea).std()
final_combo['LogLivArea'] = (np.log(x) - mnGrLivArea) / stdGrLivArea

In [76]:
final_combo

,test_norm_LotArea,test_norm_Age,test_norm_GarageArea,BsmtFullBath,KitchenAbvGr,GarageType,Kitchen,Fireplace,ExterQ,BsmtQ,HeatingQ,n_toilets,LogLivArea
0,0.523506,0.555439,1.175736,0.0,1,1,3,0,3,3,3,1.0,-1.842024
1,1.042896,0.634372,-0.696247,0.0,1,1,4,0,3,3,3,2.0,0.642691
2,0.961349,-0.791237,0.099547,0.0,1,1,3,3,3,4,4,3.0,0.836664
3,0.163714,-0.848144,0.045255,0.0,1,1,4,4,3,3,5,3.0,0.587051
4,-1.217149,-0.532619,0.207413,0.0,1,1,4,0,4,4,5,2.0,-0.376637
5,0.168756,-0.581279,-0.091578,0.0,1,1,3,3,3,4,4,3.0,-0.187722
6,-0.325547,-0.532619,-0.183725,1.0,1,1,3,0,3,4,5,3.0,0.482388
7,-0.216535,-0.848144,-0.309382,0.0,1,1,3,4,3,4,4,3.0,-0.516487
8,0.208853,-0.439228,0.207413,1.0,1,1,4,1,3,4,4,3.0,0.292933
9,-0.217043,0.298173,0.292159,1.0,1,1,3,0,3,3,3,2.0,-0.137690
